# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('../data_files/sp_500_stocks.csv')
from secrets_api import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2807170894883,
 'week52high': 183.06,
 'week52low': 133.14,
 'week52highSplitAdjustOnly': 187.72,
 'week52lowSplitAdjustOnly': 135.44,
 'week52change': 0.1640776098370423,
 'sharesOutstanding': 16583850200,
 'float': 0,
 'avg10Volume': 63301241,
 'avg30Volume': 65715318,
 'day200MovingAvg': 163.87,
 'day50MovingAvg': 164.69,
 'employees': 153265,
 'ttmEPS': 6.11,
 'ttmDividendRate': 0.9203405106892166,
 'dividendYield': 0.005475659309693653,
 'nextDividendDate': '',
 'exDividendDate': '2022-08-01',
 'nextEarningsDate': '2022-10-24',
 'peRatio': 27.852762694167975,
 'beta': 1.269155329736129,
 'maxChangePercent': 65.71666973396746,
 'year5ChangePercent': 3.636249465825985,
 'year2ChangePercent': 0.3965872757039364,
 'year1ChangePercent': 0.15538571734306433,
 'ytdChangePercent': -0.03962591644211612,
 'month6ChangePercent': 0.03546343293027608,
 'month3ChangePercent': 0.21532569348259178,
 'month1ChangePercent': 0.11423524091949981,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.15538571734306433

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for symbol in symbol_groups:
    symbol_strings.append(','.join(symbol))

print(symbol_strings)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [22]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], 
            index=my_columns),
            ignore_index=True)

print(df)

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  142.49             -0.215975                     N/A
1      AAL   14.69             -0.289392                     N/A
2      AAP  188.64             -0.128644                     N/A
3     AAPL  177.04              0.159476                     N/A
4     ABBV  145.34               0.20844                     N/A
..     ...     ...                   ...                     ...
496    YUM  119.51             -0.118041                     N/A
497    ZBH  117.19             -0.261162                     N/A
498   ZBRA  340.45             -0.435801                     N/A
499   ZION   59.83              0.036146                     N/A
500    ZTS  171.21             -0.189815                     N/A

[501 rows x 4 columns]


/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1397509904.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/139

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [29]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True)
print(df)

    index Ticker    Price One-Year Price Return Number of Shares to Buy
0     273     LB    82.34              2.283012                     N/A
1     355    OXY    75.24              2.134976                     N/A
2     147    DVN    76.22               1.76418                     N/A
3      89     CF   120.10              1.658021                     N/A
4      42    APA    41.70              1.299051                     N/A
5     315    MRO    27.10              1.268755                     N/A
6     109    COP   112.13              1.102015                     N/A
7     159    EOG   127.59              0.978923                     N/A
8     312    MOS    62.29              0.965081                     N/A
9     465    VLO   126.52              0.964647                     N/A
10    225    HRB    47.83              0.948641                     N/A
11    297   MCHP    73.23              0.926624                     N/A
12    174   FANG   142.38              0.918176                 

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [33]:
def get_portfolio_size():
    portfolio_size = None
    invalid_val = True
    while portfolio_size is None or invalid_val:
        portfolio_size = input('How big is your portfolio? ')
        try:
            val = float(portfolio_size)
            invalid_val = False
        except:
            print('Please enter a Number (Int or Float).')
        
    return val

portfolio_size = get_portfolio_size()
print(portfolio_size)

How big is your portfolio? 10000
10000.0


In [35]:
position_size = float(portfolio_size/len(df.index))
for i in range(0, len(df)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[i, 'Price'])

df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,82.34,2.283012,2
1,355,OXY,75.24,2.134976,2
2,147,DVN,76.22,1.76418,2
3,89,CF,120.10,1.658021,1
4,42,APA,41.70,1.299051,4
5,315,MRO,27.10,1.268755,7
6,109,COP,112.13,1.102015,1
7,159,EOG,127.59,0.978923,1
8,312,MOS,62.29,0.965081,3
9,465,VLO,126.52,0.964647,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [125]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            hqm_df = hqm_df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ]
            , index=hqm_columns),
            ignore_index=True)
            
hqm_df
    

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_35267/1660051289.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(
/var/folders/tl/b31rph5s7qz2n1_v

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.25,N/A,-0.218653,N/A,0.021274,N/A,0.13842,N/A,0.097783,N/A,N/A
1,AAL,14.34,N/A,-0.287091,N/A,-0.179126,N/A,-0.11079,N/A,0.049547,N/A,N/A
2,AAP,174.24,N/A,-0.12986,N/A,-0.105601,N/A,-0.015449,N/A,-0.047251,N/A,N/A
3,AAPL,166.93,N/A,0.15264,N/A,0.035661,N/A,0.21244,N/A,0.113957,N/A,N/A
4,ABBV,140.91,N/A,0.205365,N/A,-0.052101,N/A,-0.077842,N/A,-0.073191,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.56,N/A,-0.118189,N/A,-0.051823,N/A,0.01603,N/A,-0.002061,N/A,N/A
497,ZBH,111.11,N/A,-0.260375,N/A,-0.11286,N/A,-0.060333,N/A,0.039082,N/A,N/A
498,ZBRA,312.12,N/A,-0.442298,N/A,-0.216534,N/A,0.024027,N/A,0.015,N/A,N/A
499,ZION,58.85,N/A,0.037129,N/A,-0.189612,N/A,0.051307,N/A,0.067943,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [126]:
# Cleaning. I have 'None' values...

for time_period in time_periods:
    hqm_df[f'{time_period} Price Return'] = hqm_df[f'{time_period} Price Return'].apply(lambda x: x if x is not None else 0.0)

    for val in hqm_df[f'{time_period} Price Return']:
        if val is None:
            print('!!')

In [127]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])/100
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.25,N/A,-0.218653,0.193613,0.021274,0.644711,0.138420,0.842315,0.097783,0.754491,N/A
1,AAL,14.34,N/A,-0.287091,0.117764,-0.179126,0.173653,-0.110790,0.083832,0.049547,0.451098,N/A
2,AAP,174.24,N/A,-0.129860,0.355289,-0.105601,0.333333,-0.015449,0.289421,-0.047251,0.065868,N/A
3,AAPL,166.93,N/A,0.152640,0.782435,0.035661,0.672655,0.212440,0.922156,0.113957,0.824351,N/A
4,ABBV,140.91,N/A,0.205365,0.820359,-0.052101,0.44511,-0.077842,0.123752,-0.073191,0.033932,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.56,N/A,-0.118189,0.381238,-0.051823,0.447106,0.016030,0.437126,-0.002061,0.191617,N/A
497,ZBH,111.11,N/A,-0.260375,0.145709,-0.112860,0.321357,-0.060333,0.167665,0.039082,0.381238,N/A
498,ZBRA,312.12,N/A,-0.442298,0.043912,-0.216534,0.121756,0.024027,0.47505,0.015000,0.269461,N/A
499,ZION,58.85,N/A,0.037129,0.634731,-0.189612,0.151697,0.051307,0.58483,0.067943,0.566866,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [128]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
        
        hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
        
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.25,N/A,-0.218653,0.193613,0.021274,0.644711,0.138420,0.842315,0.097783,0.754491,0.608782
1,AAL,14.34,N/A,-0.287091,0.117764,-0.179126,0.173653,-0.110790,0.083832,0.049547,0.451098,0.206587
2,AAP,174.24,N/A,-0.129860,0.355289,-0.105601,0.333333,-0.015449,0.289421,-0.047251,0.065868,0.260978
3,AAPL,166.93,N/A,0.152640,0.782435,0.035661,0.672655,0.212440,0.922156,0.113957,0.824351,0.800399
4,ABBV,140.91,N/A,0.205365,0.820359,-0.052101,0.44511,-0.077842,0.123752,-0.073191,0.033932,0.355788
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.56,N/A,-0.118189,0.381238,-0.051823,0.447106,0.016030,0.437126,-0.002061,0.191617,0.364271
497,ZBH,111.11,N/A,-0.260375,0.145709,-0.112860,0.321357,-0.060333,0.167665,0.039082,0.381238,0.253992
498,ZBRA,312.12,N/A,-0.442298,0.043912,-0.216534,0.121756,0.024027,0.47505,0.015000,0.269461,0.227545
499,ZION,58.85,N/A,0.037129,0.634731,-0.189612,0.151697,0.051307,0.58483,0.067943,0.566866,0.484531


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [129]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,48.08,N/A,0.910686,0.98004,0.989693,1.0,0.380178,0.996008,0.219493,0.97006,0.986527
1,CF,122.64,N/A,1.671863,0.994012,0.511410,0.99002,0.173783,0.89022,0.315463,0.996008,0.967565
2,ALB,290.34,N/A,0.277824,0.876248,0.551705,0.992016,0.257106,0.966068,0.305860,0.994012,0.957086
3,AIV,9.00,N/A,0.353757,0.904192,0.275266,0.958084,0.525190,1.0,0.213274,0.962076,0.956088
4,UNM,39.32,N/A,0.559082,0.956088,0.443238,0.986028,0.159845,0.878244,0.240585,0.982036,0.950599
5,CAH,70.20,N/A,0.399996,0.934132,0.306827,0.968064,0.217584,0.932136,0.207528,0.956088,0.947605
6,GWW,587.63,N/A,0.365186,0.922156,0.235506,0.94012,0.248734,0.954092,0.215055,0.968064,0.946108
7,COG,22.76,N/A,0.355695,0.908184,0.184092,0.898204,0.266569,0.974052,0.251483,0.984032,0.941118
8,OXY,76.12,N/A,2.045233,0.998004,0.933507,0.998004,0.124158,0.802395,0.158259,0.912176,0.927645
9,ADP,253.38,N/A,0.272028,0.872255,0.279318,0.962076,0.218463,0.934132,0.184141,0.94012,0.927146


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [130]:
portfolio_size = get_portfolio_size()

How big is your portfolio? 10000


In [131]:
position_size = float(portfolio_size/len(hqm_df.index))
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,48.08,4,0.910686,0.98004,0.989693,1.0,0.380178,0.996008,0.219493,0.97006,0.986527
1,CF,122.64,1,1.671863,0.994012,0.511410,0.99002,0.173783,0.89022,0.315463,0.996008,0.967565
2,ALB,290.34,0,0.277824,0.876248,0.551705,0.992016,0.257106,0.966068,0.305860,0.994012,0.957086
3,AIV,9.00,22,0.353757,0.904192,0.275266,0.958084,0.525190,1.0,0.213274,0.962076,0.956088
4,UNM,39.32,5,0.559082,0.956088,0.443238,0.986028,0.159845,0.878244,0.240585,0.982036,0.950599
5,CAH,70.20,2,0.399996,0.934132,0.306827,0.968064,0.217584,0.932136,0.207528,0.956088,0.947605
6,GWW,587.63,0,0.365186,0.922156,0.235506,0.94012,0.248734,0.954092,0.215055,0.968064,0.946108
7,COG,22.76,8,0.355695,0.908184,0.184092,0.898204,0.266569,0.974052,0.251483,0.984032,0.941118
8,OXY,76.12,2,2.045233,0.998004,0.933507,0.998004,0.124158,0.802395,0.158259,0.912176,0.927645
9,ADP,253.38,0,0.272028,0.872255,0.279318,0.962076,0.218463,0.934132,0.184141,0.94012,0.927146


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [132]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [133]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [134]:
column_formats = { 
                'A': ['Ticker', string_template],
                'B': ['Price', dollar_template],
                'C': ['Number of Shares to Buy', integer_template],
                'D': ['One-Year Price Return', percent_template],
                'E': ['One-Year Return Percentile', percent_template],
                'F': ['Six-Month Price Return', percent_template],
                'G': ['Six-Month Return Percentile', percent_template],
                'H': ['Three-Month Price Return', percent_template],
                'I': ['Three-Month Return Percentile', percent_template],
                'J': ['One-Month Price Return', percent_template],
                'K': ['One-Month Return Percentile', percent_template],
                'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [135]:
writer.save()